# Query MapRDB Using Spark SQL
This notebook is still a work in progress, and requires some "unsupported" configurations in the mapr cluster before it can run against it. At a high level, i'm using mapr-livy (running at the testuser) on a cluster node to submit spark jobs in the cluster, and load the data from maprdb. I also had to pass in hive-site.xml to obtain the connection to the hive metastore, file which contains plaintext passwords. For now, please use this notebook as a conceptual demo only. 

In [8]:
%load_ext sparkmagic.magics


The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics


In [9]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Added endpoint http://172.16.9.42:8998
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1523343062059_0002,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Verify connection to hive mestatore

you should be able to see existing tables in hive

In [27]:
%%spark -s testing -c sql -o localvar 
show tables


,database,tableName,isTemporary
0,default,all_blocks_parquet_hive,False
1,default,all_blocks_sparkie,False
2,default,customers,False
3,default,newtable,False
4,default,sample_07,False
5,default,sample_08,False
6,default,table1,False
7,default,web_logs,False


### Load all_transactions_table from maprdb into a spark dataframe, and register as a temporary view

In [28]:
%%spark -s testing
df = spark.loadFromMapRDB("/user/testuser/eth/all_transactions_table")
df.createOrReplaceTempView("all_transactions_view")
df.count()
#df.printSchema()
#sqlDF = spark.sql("SELECT * from all_transactions_view")
#sqlDF.show()

1029858

In [29]:
%%spark -s testing -c sql -o localvar
create table all_transactions_hive_table as select * from all_transactions_view

""


In [32]:
%%spark -s testing -c sql -o localvar
select count(*) from all_transactions_hive_table

,count(1)
0,1029858


In [33]:
%%spark
df = spark.loadFromMapRDB("/user/testuser/all_txs")
df.createOrReplaceTempView("all_txs_view2")
sqlDF = spark.sql("SELECT * from all_txs_view2")
df.write.parquet("/user/testuser/all_parquet_x2")

In [ ]:
%manage_spark

In [ ]:
%%spark
sc.version